## Dev version of integrating arima in stats models with sklearn various types of grid search to optimize arima hyperparameters.

In [1]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels as stm
import seaborn as sns
import pandas as pd
from sklearn.model_selection import GridSearchCV

import statsmodels.api as sm
from statsmodels.graphics.api import qqplot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARMA
from scipy.optimize import brute

/home/dan/miniconda2/envs/basic_ml/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [13]:
from statsmodels.tsa.stattools import adfuller

In [2]:
# Get the data for this test.
df = pd.read_csv('../../data_sets/air_passengers/AirPassengers.csv')

In [3]:
df.head()

,Month,#Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [4]:
df.rename(index=str, columns={'#Passengers': 'passengers', 'Month': 'month'}, inplace=True)

In [5]:
df.head()

,month,passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [6]:
# reset index.
df.index = pd.to_datetime(df['month'])
df.drop(['month'], axis=1, inplace=True)

In [7]:
df.head()

,passengers
month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


In [9]:
# This will be our test example.  We'll need four functions.  One to resample the data if
# necessary.  Another to check for stationarity.  Then another to search the space of 
# transformations to find the best stationary time series if possible.  We'll need to be
# careful about seasonality here...and then one more to find the optimal fit parameters.
# We'll probably actually want a fifth function to invert the transformations and then test
# the fit quality....

# At the end of the day, we'll turn this all into a custom class, but first let's get the 
# functions working.

## Look for outliters

In [10]:
# A simple outlier detection function.
def remove_outliers_simple(df, standard_deviations):
    # There should only be one numerical column
    col = df.columns.values.tolist()[0]
    # calculate the z statistic
    z = pd.DataFrame((df[col] - df[col].mean()).div(df[col].std()))
    # Identify outliers based on input number of standard_deviations
    outliers = z > standard_deviations
    # filter
    df_out = df[~outliers.any(axis=1)]
    
    return df_out

## Resample function

In [12]:
def re_sample_custom(df, unit, method='linear'):
    resampled = df.resample(unit)
    interpolated = resampled.interpolate(method=method)
    
    return interpolated

## Check for stationarity

In [15]:
def test_stationarity(timeseries):
    # TODO: add a recommendation to return...
    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window=12)
    rolstd = pd.rolling_std(timeseries, window=12)

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

## Search for a stationary series

In [ ]:
# We need a function to try out a few transformations if not stationary...we should merge this
# with the above function.

In [ ]:
class OutlierRemovalFeatureSelectionRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, outlier_detector, regressor, feature_selector, use_feature_selection, 
                 use_tree_selection=False, select_alg=None, use_outlier_detection=False):
        self.outlier_detector = outlier_detector
        self.regressor = regressor
        self.feature_selector = feature_selector
        self.use_tree_selection = use_tree_selection
        self.select_alg = select_alg
        self.use_outlier_detection = use_outlier_detection
        self.use_feature_selection = use_feature_selection

    def fit(self, X, y):
        if self.use_outlier_detection:
            X_resample, y_resample = self.resample(X, y)
        else:
            X_resample = X.copy()
            y_resample = y.copy()
        if self.use_feature_selection:
            X_resample_features = self.feature_select(X_resample, y_resample)
        else:
            X_resample_features = X_resample.copy()
        # Generate the validation set for early stopping
        X_train, X_valid, y_train, y_valid = self.make_validation_set(X_resample_features, y_resample)
        # set the early_stopping rounds to be about 10% the number of epochs. Otherwise risk under fitting...
        self.regressor_ = clone(self.regressor).fit(X_train, y_train, eval_metric='mae',
                                                    eval_set=[[X_valid, y_valid]], early_stopping_rounds=50, verbose=False)
        return self

    def predict(self, X):
        if self.use_feature_selection:
            X_red = self.feature_selector.transform(X)
        else:
            X_red = X.copy()
        return self.regressor_.predict(X_red)

    def resample(self, X, y):
        self.outlier_detector_ = clone(self.outlier_detector)
        y_shape = y.reshape((y.shape[0], 1))
        X_mat = np.hstack((X, y_shape))
        self.outlier_detector = self.outlier_detector_.fit(X_mat)
        mask = self.outlier_detector.predict(X_mat) == 1
        return X[mask], y[mask]

    def feature_select(self, X, y):
        # This will run the feature selection component.
        if self.use_tree_selection:
            # THIS DOESN'T WORK! DON'T USE IT YET!
            self.feature_selector_ = clone(self.feature_selector)
            # self.select_alg_ = clone(self.select_alg)
            self.feature_selector = SelectFromModel(self.feature_selector_.fit(X, y))
            # self.feature_selector = self.select_alg(self.feature_selector, prefit=True)
            # self.feature_selector = SelectFromModel(self.feature_selector, prefit=True)
            feature_selected_X = self.feature_selector.transform(X)
        else:
            self.feature_selector_ = clone(self.feature_selector)
            self.feature_selector = self.feature_selector_.fit(X, y)
            feature_selected_X = self.feature_selector.transform(X)
        return feature_selected_X

    def make_validation_set(self, X, y):
        # This will generate the validation set for the xgboost regressor.
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)
        return X_train, X_valid, y_train, y_valid